Check the data and create auxiliary text files.

In [2]:
import pandas as pd
import re
import geonamescache
import unidecode
import os
import sys
import difflib

filename = "headlines.txt"

f = open(filename, "r")
theHeadlines = f.readlines()
theHeadlines.sort()
f.close()
nHeadlines = len(theHeadlines)
print("n headlines: {}".format(nHeadlines))

f = open("headlines-sorted.txt", "w")
for k in theHeadlines:
    f.write(k)
f.close()

gc = geonamescache.GeonamesCache()
countriesDict = gc.get_countries()
print(len(countriesDict))

citiesDict = gc.get_cities()
nCities = len(citiesDict)
print("n cities: {}".format(nCities))

n headlines: 650
252
n cities: 24336


Create list of cities with and without special characters.

In [3]:
theCities = []
for k in citiesDict.keys():
    theCities.append(citiesDict[k]["name"])
theCities.sort()

theCitiesNoAccents = []
for k in theCities:
    theCitiesNoAccents.append(unidecode.unidecode(k))


Write auxiliary file

In [4]:
f = open("cityNames.txt", "w")
for k in theCities:
    f.write(k + "\n")
f.close()

f = open("cityNames-noAccents.txt", "w")
for k in theCitiesNoAccents:
    f.write(k + "\n")
f.close()

Write auxiliary file

In [5]:
theHeadlinesNoAccents = []
for k in theHeadlines:
    theHeadlinesNoAccents.append(unidecode.unidecode(k))
f = open("headlines-noAccents.txt", "w")
for k in theHeadlinesNoAccents:
    f.write(k)
f.close()

Compare the files.

In [6]:
text1 = open("headlines-sorted.txt")
text2 = open("headlines-noAccents.txt")
sys.stdout.writelines(difflib.context_diff(theHeadlines, theHeadlinesNoAccents, n=0))

*** 
--- 
***************
*** 29 ****
! BREAKING – Zika in Missoula
--- 29 ----
! BREAKING - Zika in Missoula
***************
*** 340 ****
! Panama City’s first Zika related death
--- 340 ----
! Panama City's first Zika related death
***************
*** 573 ****
! Zika alert – Manila now threatened
--- 573 ----
! Zika alert - Manila now threatened
***************
*** 590 ****
! Zika case recorded in Tunapuna »
--- 590 ----
! Zika case recorded in Tunapuna >>


Simple try to find matches, it obviously is not correct.

In [7]:
headlineCityCandidates = {}
n = 0
for oneArticle in theHeadlinesNoAccents:
    headlineCityCandidates[oneArticle] = []
    for oneCity in theCitiesNoAccents:
        if oneCity in oneArticle:
            n += 1
            headlineCityCandidates[oneArticle].append(oneCity)
print("In {} headlines and {} cities we have {} matches".format(nHeadlines,
        nCities, n))

# eval how many headlines had a given amount of matches
headlinesCityMatchCount = {}
for k in headlineCityCandidates.keys():
    n = len(headlineCityCandidates[k])
    if n not in headlinesCityMatchCount:
        headlinesCityMatchCount[n] = 0
    headlinesCityMatchCount[n] += 1
print(headlinesCityMatchCount)

# to be more legible print in descending order of the matches
nOcorrencias = list(headlinesCityMatchCount.keys())
nOcorrencias.sort()
for k in nOcorrencias:
    print("{}:{} ".format(k, headlinesCityMatchCount[k]), end=" ")
print("")

In 650 headlines and 24336 cities we have 1505 matches
{4: 53, 1: 216, 7: 12, 2: 192, 3: 98, 6: 17, 0: 24, 5: 27, 8: 7, 10: 1, 9: 1}
0:24  1:216  2:192  3:98  4:53  5:27  6:17  7:12  8:7  9:1  10:1  


Check matches that failed.

In [9]:
print("The {} headlines with no city detected:".format(headlinesCityMatchCount[0]))
for k in headlineCityCandidates.keys():
    if len(headlineCityCandidates[k]) == 0:
        print(k, end="")


The 24 headlines with no city detected:
Carnival under threat in Rio De Janeiro due to Zika outbreak
Chikungunya has not Left Pismo Beach
Fort Belvoir tests new cure for Hepatitis C
Longwood volunteers spreading Zika awareness
Louisiana Zika cases up to 26
Maka City Experiences Influenza Outbreak
More Zika patients reported in Mcallen
More people in Huron are infected with Dengue every year
More people in Mclean are infected with Hepatitis A every year
Outbreak of Zika in Alvorada
Outbreak of Zika in Destin
Outbreak of Zika in Hutchins
Rumors about Influenza Spreading in Dobbs Ferry have been Refuted
Schools in Coamo Closed Due to Rhinovirus Outbreak
Spike of Hepatitis E Cases in Morehead City
Spike of Meningitis Cases in Druid Hills
Yulee takes a hit from Spreading Sickness
Zika Troubles come to Padre Las Casas
Zika arrives in Dangriga
Zika case reported in Antioquia
Zika case reported in Oton
Zika infects pregnant woman in Cebu
Zika spreads to La Joya
Zika symptoms spotted in Quixere

Check city with largest number of matches.

In [10]:
maxOcorrencias = nOcorrencias[-1] # last list item in an ascending sorted list
print("The {} headlines where more matches of cities were detected:".format(
            headlinesCityMatchCount[maxOcorrencias]))
for k in headlineCityCandidates.keys():
    if len(headlineCityCandidates[k]) == maxOcorrencias:
        print(k, end="")
        print("   Match: {}".format(headlineCityCandidates[k]))


The 1 headlines where more matches of cities were detected:
Lower Hospitalization in Richmond after Mumps Vaccine becomes Mandatory
   Match: ['Ho', 'Man', 'Richmond', 'Richmond', 'Richmond', 'Richmond', 'Richmond', 'Richmond', 'Richmond', 'Vac']


Create list sorted by descending order of city name's length.

In [11]:
# using a lambda because it is negative it sorts in descending order
theCities.sort(key = lambda city: -len(city)) 
print("The 10 cities with longer names: {}".format(theCities[0:10]))
print("...")
print("The 10 cities with shorter names: {}".format(theCities[-10:]))

theCitiesNoAccents = []
for k in theCities:
    theCitiesNoAccents.append(unidecode.unidecode(k).lower())

theHeadlinesNoAccents = []
for k in theHeadlines:
    theHeadlinesNoAccents.append(unidecode.unidecode(k).lower())

The 10 cities with longer names: ['Chak Two Hundred Forty-nine Thal Development Authority', 'Dolores Hidalgo Cuna de la Independencia Nacional', 'Ampliación San Mateo (Colonia Solidaridad)', 'Licenciado Benito Juárez (Campo Gobierno)', 'Sant Pere, Santa Caterina i La Ribera', 'Nanchital de Lázaro Cárdenas del Río', 'Palikir - National Government Center', 'San Fernando del Valle de Catamarca', 'San Martin Texmelucan de Labastida', "el Camp d'en Grassot i Gràcia Nova"]
...
The 10 cities with shorter names: ['Ōzu', 'Ōzu', 'Bo', 'Ho', 'Of', 'Pô', 'Un', 'Wa', 'Ōi', 'Ūn']


Create list of compiled regular expressions.

In [12]:
theCitiesNoAccentsRegExpr = []
for k in theCitiesNoAccents:
    theCitiesNoAccentsRegExpr.append(re.compile(r"([^a-zA-Z]|^)" + 
        k + r"([^a-zA-Z]|$)"))

Detect cities.

In [13]:
nFailures = 0
headlineIndexes = []
nCities = len(theCitiesNoAccentsRegExpr)
failedMatches = []
for idx in range(nHeadlines):
    oneHeadline = theHeadlinesNoAccents[idx]
    theMatches = []
    for k in range(nCities):
        if theCitiesNoAccentsRegExpr[k].search(oneHeadline) != None:
            # print(k, oneHeadline, theCities[k])
            if theCities[k] == "Of":
                if "Of" not in oneHeadline.strip().split():
                    print("Suspect match ignored: {} -> of".format(theHeadlines[idx].strip()))
                    continue
            theMatches.append(k)
            break # we only want the 1st match
    headlineIndexes.append(theMatches)
    if len(theMatches) == 0:
        nFailures += 1
        failedMatches.append(theHeadlines[idx])
print("{} match failures".format(nFailures))
for k in failedMatches:
    print("FAILED: {}".format(k), end="")

for k in [0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 607, 649]:
    if len(headlineIndexes[k]) != 0:
        print("Headline {}: {} -> {}".format(k, theHeadlines[k], theCities[headlineIndexes[k][0]]))
    else:
        print("{} {} -> {}".format(k, theHeadlines[k], "*** NO MATCH"))


Suspect match ignored: Outbreak of Zika in Alvorada -> of
Suspect match ignored: Outbreak of Zika in Destin -> of
Suspect match ignored: Outbreak of Zika in Hutchins -> of
Suspect match ignored: Spike of Hepatitis E Cases in Morehead City -> of
Suspect match ignored: Spike of Meningitis Cases in Druid Hills -> of
Suspect match ignored: The Spread of Chikungunya in Davidson has been Confirmed -> of
37 match failures
FAILED: Bronchitis Outbreak in Manhasset
FAILED: Chikungunya has not Left Pismo Beach
FAILED: Fort Belvoir tests new cure for Hepatitis C
FAILED: Greenwich Establishes Zika Task Force
FAILED: Gympie Patient in Critical Condition after Contracting Chlamydia
FAILED: Hillsborough uses innovative trap against Zika 20 minutes ago
FAILED: Longwood volunteers spreading Zika awareness
FAILED: Louisiana Zika cases up to 26
FAILED: Maka City Experiences Influenza Outbreak
FAILED: Malaria Exposure in Sussex
FAILED: Martinsville tests new cure for Measles
FAILED: Measles Hits Davos
FAIL

Create list of cities and countries.

In [14]:
headlineNoLinebreak = []
headlineCity = []
headlineCountry = []

for k in range(nHeadlines):
    headlineNoLinebreak.append(theHeadlines[k].strip())		
    if len(headlineIndexes[k]) == 0:
        headlineCity.append(None)
        headlineCountry.append(None)
    else:
        headlineCity.append(theCities[headlineIndexes[k][0]])
        theCity = gc.get_cities_by_name(theCities[headlineIndexes[k][0]])
        theCityDict = theCity[0]
        theCityKey = list(theCityDict.keys())[0] 
        countryCode = theCityDict[theCityKey]["countrycode"]
        headlineCountry.append(countriesDict[countryCode]["name"])

for k in range(10):
    print("{}: {} -> {} ({})".format(k, theHeadlines[k].strip(),
            headlineCity[k], headlineCountry[k]))


0: 18 new Zika Cases in Bogota -> Bogotá (Colombia)
1: 19 new Zika Cases in Sengkang -> Sengkang (Indonesia)
2: Alameda Residents Recieve Rabies vaccine -> Alameda (United States)
3: Albany Residents Recieve Respiratory Syncytial Virus vaccine -> Albany (Australia)
4: Antipolo under threat from Zika Virus -> Antipolo (Philippines)
5: Arhus is infested with Bronchitis -> Århus (Denmark)
6: Arvada is infested with Syphilis -> Arvada (United States)
7: Authorities a Miami -> Miami (United States)
8: Authorities are Worried about the Spread of Bronchitis in Silver Spring -> Silver Spring (United States)
9: Authorities are Worried about the Spread of Chickenpox in Hemet -> Hemet (United States)


Create dataframe.

In [15]:
headlineCityCountry = {
	'headline': headlineNoLinebreak, 
	'countries': headlineCountry,
	'cities': headlineCity
}
df = pd.DataFrame(headlineCityCountry)
print(df.head(10))


                                            headline      countries  \
0                        18 new Zika Cases in Bogota       Colombia   
1                      19 new Zika Cases in Sengkang      Indonesia   
2           Alameda Residents Recieve Rabies vaccine  United States   
3  Albany Residents Recieve Respiratory Syncytial...      Australia   
4              Antipolo under threat from Zika Virus    Philippines   
5                  Arhus is infested with Bronchitis        Denmark   
6                   Arvada is infested with Syphilis  United States   
7                                Authorities a Miami  United States   
8  Authorities are Worried about the Spread of Br...  United States   
9  Authorities are Worried about the Spread of Ch...  United States   

          cities  
0         Bogotá  
1       Sengkang  
2        Alameda  
3         Albany  
4       Antipolo  
5          Århus  
6         Arvada  
7          Miami  
8  Silver Spring  
9          Hemet  
